In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_rows = 999

In [2]:
df = pd.read_csv('dod_all_states.csv')

In [3]:
df.head()

,State,Agency Name,NSN,Item Name,Quantity,UI,Acquisition Value,DEMIL Code,DEMIL IC,Ship Date,Station Type
0,AL,ABBEVILLE POLICE DEPT,2540-01-565-4700,BALLISTIC BLANKET KIT,10,Kit,15871.59,D,1.0,2018-01-30 00:00:00.000,State
1,AL,ABBEVILLE POLICE DEPT,1240-DS-OPT-SIGH,OPTICAL SIGHTING AND RANGING EQUIPMENT,1,Each,245.88,D,NaN,2016-06-02 00:00:00.000,State
2,AL,ABBEVILLE POLICE DEPT,2355-01-553-4634,MINE RESISTANT VEHICLE,1,Each,658000.00,C,1.0,2016-11-09 00:00:00.000,State
3,AL,ABBEVILLE POLICE DEPT,1240-01-411-1265,"SIGHT,REFLEX",9,Each,333.00,D,1.0,2016-09-14 00:00:00.000,State
4,AL,ABBEVILLE POLICE DEPT,5855-01-577-7174,"ILLUMINATOR,INFRARED",10,Each,926.00,D,1.0,2017-03-28 00:00:00.000,State


In [4]:
df.isnull().sum()

State                   0
Agency Name             0
NSN                     0
Item Name               0
Quantity                0
UI                      0
Acquisition Value       0
DEMIL Code              0
DEMIL IC             9910
Ship Date               0
Station Type            0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130958 entries, 0 to 130957
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   State              130958 non-null  object 
 1   Agency Name        130958 non-null  object 
 2   NSN                130958 non-null  object 
 3   Item Name          130958 non-null  object 
 4   Quantity           130958 non-null  int64  
 5   UI                 130958 non-null  object 
 6   Acquisition Value  130958 non-null  float64
 7   DEMIL Code         130958 non-null  object 
 8   DEMIL IC           121048 non-null  float64
 9   Ship Date          130958 non-null  object 
 10  Station Type       130958 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 11.0+ MB


In [6]:
df.columns = df.columns.str.replace(' ','_')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130958 entries, 0 to 130957
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   State              130958 non-null  object 
 1   Agency_Name        130958 non-null  object 
 2   NSN                130958 non-null  object 
 3   Item_Name          130958 non-null  object 
 4   Quantity           130958 non-null  int64  
 5   UI                 130958 non-null  object 
 6   Acquisition_Value  130958 non-null  float64
 7   DEMIL_Code         130958 non-null  object 
 8   DEMIL_IC           121048 non-null  float64
 9   Ship_Date          130958 non-null  object 
 10  Station_Type       130958 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 11.0+ MB


In [8]:
df.DEMIL_IC.value_counts()

1.0    112444
3.0      5406
7.0      2891
6.0       250
0.0        42
4.0        14
5.0         1
Name: DEMIL_IC, dtype: int64

https://csp.colorado.gov/sites/csp/files/documents/13_DEMIL%20Codes.pdf
1. 1 - DEMIL code reviewed by DCMO, rec. adopted by ICP, or no DEMIL code change rec.
2. 3 - Critical FSC/FSG MLI or sensitive CCLI requires mutilation worldwide
3. 7 - Forced concurrence, DCMO has forced a DEMIL code change in FLIS. ICP has not responded to collaboration request (over 90 days old) or failed to update the DEMIL code in the ICPs legacy system.
4. 6 - Non-Critical FSC/FSG MLI or Non-Sensitive CCLI. Requires mutilation overseas
5. 0 - DEMIL code reviewed by DCMO. Rec. DEMIL code and current ICP DEMIL code are not equal. Presently in collaboration cycle.
6. 4 - DEMIL code could not be validated - insufficient technical data available.
7. 5 - Item reviewed and coded by Service/Agency ICP - without DCMO collaboration, or service/agency ICP changed the DEMIL code prior to completion of IC-0 review/collaboration cycle

In [9]:
df.DEMIL_Code.value_counts()

D    93478
A    18312
C     7012
F     6198
Q     5658
B      156
E      144
Name: DEMIL_Code, dtype: int64

https://www.dla.mil/Portals/104/Documents/DispositionServices/ddsr/userguide/demilcodes.html
1. D - DEMIL required. Destroy items and components to prevent restoration or reprair to a usable condition. 
2. A - Non-USML/Non-CCL - No DEMIL or DOD TSC required. 
3. C - USML items - DEMIL required, remove and demilitarize installed key points
4. F - Special instructions for DEMIL (environmental, safety, health hazards)
5. Q - Commerce Control List Items (CCLI) under jurisdiction of Bureau of Inudstry and Security, US Dep. of Commerce. 
6. B - USML items - mutilation to the point of scrap required worldwide
7. E - DDPO, DoD Demilitarization Program Office, generally treated as DEMIL code 'D'

NOTE:
DEMIL IC 1, 3, 6 or 7 signifies that DCMO has verified DEMIL code as accurate and locked it in FLIS. DoD
DEMIL Manual (DoD 4160.21-M-1) provides instructions regarding DEMIL Code changes after the lock is installed.

### Handling DEMIL_IC nulls:
Due to the sensitive nature of this kind of data, in terms of mis-attributing the status of something, I am very wary on retaining data that has nulls. However, looking at the codes, their meanings, and the fact that this equipment was intentionally sent out I think it is appropriate to assign the value of '1' to the nulls. The DEMIL code largely does not seem to be the most crucial descriptor of the data, is not going to be the target variable, and is mostly only a handful of values anyways. But, I do want to reiterate that this was not done without consideration for the consequences of inaccurately assigning this value to datapoints. 

Given the frequency of the assigned value of 1, the fact that this gear was clearly handled with intent (aligning with that IC value), and the above note (regarding 1,3,6,7) I will be assigning the nulls as '1'. However, this will be recalled and considered when interacting with the data moving forward if it becomes a crucial element of the analysis. 

In [10]:
# 
112444 + 9910

122354

In [11]:
df.DEMIL_IC = df.DEMIL_IC.fillna(1)

In [12]:
df.DEMIL_IC.value_counts()

1.0    122354
3.0      5406
7.0      2891
6.0       250
0.0        42
4.0        14
5.0         1
Name: DEMIL_IC, dtype: int64

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130958 entries, 0 to 130957
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   State              130958 non-null  object 
 1   Agency_Name        130958 non-null  object 
 2   NSN                130958 non-null  object 
 3   Item_Name          130958 non-null  object 
 4   Quantity           130958 non-null  int64  
 5   UI                 130958 non-null  object 
 6   Acquisition_Value  130958 non-null  float64
 7   DEMIL_Code         130958 non-null  object 
 8   DEMIL_IC           130958 non-null  float64
 9   Ship_Date          130958 non-null  object 
 10  Station_Type       130958 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 11.0+ MB


### Nulls handled. To do for preparation:
- convert Ship_Date to datetime and set as index
- change DEMIL_IC datatype to object

In [14]:
df.Ship_Date = pd.to_datetime(df.Ship_Date)

In [15]:
df = df.set_index(df.Ship_Date)

In [16]:
df = df.sort_index()

In [17]:
df.Quantity.value_counts()

1       106745
2         4593
4         2268
3         2170
10        1932
5         1807
6         1363
20         997
8          847
7          639
12         628
15         626
25         462
9          437
30         431
50         356
14         335
11         298
16         240
13         232
40         227
18         194
100        191
17         167
24         165
19         127
35         119
21         109
22         105
23         101
60         100
27          87
28          81
26          72
32          71
75          65
34          57
45          56
42          54
31          53
200         52
29          52
36          47
38          43
150         43
37          39
48          37
70          36
80          36
33          33
44          31
41          30
43          29
39          29
120         28
46          27
52          26
90          25
300         23
49          23
55          18
63          17
47          17
250         16
54          14
61          13
51        

In [18]:
df[df.Quantity > 250]

,State,Agency_Name,NSN,Item_Name,Quantity,UI,Acquisition_Value,DEMIL_Code,DEMIL_IC,Ship_Date,Station_Type
Ship_Date,,,,,,,,,,,
2009-10-26 00:00:00.000,TN,KNOX COUNTY SHERIFF'S OFFICE,1005-00-921-5004,"MAGAZINE,CARTRIDGE",300,Each,13.59,D,1.0,2009-10-26 00:00:00.000,State
2009-10-26 00:00:00.000,TN,KNOX COUNTY SHERIFF'S OFFICE,1005-00-921-5004,"MAGAZINE,CARTRIDGE",300,Each,13.59,D,1.0,2009-10-26 00:00:00.000,State
2011-02-02 00:00:00.000,CA,LOS ANGELES POLICE DEPARTMENT,1005-00-921-5004,"MAGAZINE,CARTRIDGE",488,Each,13.59,D,1.0,2011-02-02 00:00:00.000,State
2011-03-03 00:00:00.000,CA,ORANGE COUNTY SHERIFFS DEPT,1005-01-424-5899,"SPRING,ASSEMBLY,EXTRACTOR",300,Each,0.81,Q,3.0,2011-03-03 00:00:00.000,State
2011-03-03 00:00:00.000,CA,ORANGE COUNTY SHERIFFS DEPT,1005-00-992-7287,"RING,BOLT",671,Package,2.00,Q,3.0,2011-03-03 00:00:00.000,State
2011-04-05 00:00:00.000,ME,BUREAU OF FORESTRY FOREST RANGERS,2530-01-496-8858,"PAD,TRACK SHOE",396,Each,14.52,Q,3.0,2011-04-05 00:00:00.000,State
2011-05-10 00:00:00.000,KY,STATE POLICE,1005-00-921-5004,"MAGAZINE,CARTRIDGE",774,Each,13.59,D,1.0,2011-05-10 00:00:00.000,State
2011-10-12 00:00:00.000,CA,ORANGE COUNTY SHERIFFS DEPT,5360-00-992-7301,"SPRING,HELICAL,COMPRESSION",300,Each,0.28,Q,3.0,2011-10-12 00:00:00.000,State
2011-12-06 00:00:00.000,TX,MCLENNAN COUNTY SHERIFF'S OFFICE,1005-00-921-5004,"MAGAZINE,CARTRIDGE",290,Each,13.59,D,1.0,2011-12-06 00:00:00.000,State


In [19]:
# Need for a 'total_acquisition_value' column: the acquisition column does not account for quantity of item sold

In [20]:
df.DEMIL_IC = df.DEMIL_IC.astype(int)
df.DEMIL_IC = df.DEMIL_IC.astype(object)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 130958 entries, 1980-01-01 09:07:07 to 2021-09-30 00:00:00
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   State              130958 non-null  object        
 1   Agency_Name        130958 non-null  object        
 2   NSN                130958 non-null  object        
 3   Item_Name          130958 non-null  object        
 4   Quantity           130958 non-null  int64         
 5   UI                 130958 non-null  object        
 6   Acquisition_Value  130958 non-null  float64       
 7   DEMIL_Code         130958 non-null  object        
 8   DEMIL_IC           130958 non-null  object        
 9   Ship_Date          130958 non-null  datetime64[ns]
 10  Station_Type       130958 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(8)
memory usage: 12.0+ MB


In [22]:
df['total_value'] = df.Quantity * df.Acquisition_Value

In [23]:
df.head()

,State,Agency_Name,NSN,Item_Name,Quantity,UI,Acquisition_Value,DEMIL_Code,DEMIL_IC,Ship_Date,Station_Type,total_value
Ship_Date,,,,,,,,,,,,
1980-01-01 09:07:07,NY,GREECE PD,2320-01-074-7642,"TRUCK,ARMORED",1,Each,65070.0,C,1,1980-01-01 09:07:07,State,65070.0
1990-05-03 00:00:00,MT,HIGHWAY PATROL HELENA,1005-00-589-1271,"RIFLE,7.62 MILLIMETER",1,Each,138.0,D,1,1990-05-03 00:00:00,State,138.0
1990-05-03 00:00:00,MT,HIGHWAY PATROL HELENA,1005-00-589-1271,"RIFLE,7.62 MILLIMETER",1,Each,138.0,D,1,1990-05-03 00:00:00,State,138.0
1990-05-03 00:00:00,MT,HIGHWAY PATROL HELENA,1005-00-589-1271,"RIFLE,7.62 MILLIMETER",1,Each,138.0,D,1,1990-05-03 00:00:00,State,138.0
1990-05-03 00:00:00,MT,HIGHWAY PATROL HELENA,1005-00-589-1271,"RIFLE,7.62 MILLIMETER",1,Each,138.0,D,1,1990-05-03 00:00:00,State,138.0


In [24]:
df.Station_Type.value_counts()

State    130958
Name: Station_Type, dtype: int64

In [25]:
# This is all state data so can drop Station_Type
df = df.drop(columns='Station_Type')

In [26]:
df.UI.value_counts()

Each          125920
Kit             1747
Pair             944
Set              653
Box              532
Package          447
Assembly         211
Roll             136
Dozen            121
Hundred           42
Unknown           32
Sheet             31
Reel              22
Foot              20
Case              19
Spool             16
Assortment        11
Bag                9
Thousand           9
Bale               6
Outfit             4
Yard               4
Coil               4
Plate              4
Pound              3
Board Foot         3
Bundle             2
Can                2
Cartridge          1
Pad                1
Gross              1
Bottle             1
Name: UI, dtype: int64

In [27]:
df[df.UI != 'Each']

,State,Agency_Name,NSN,Item_Name,Quantity,UI,Acquisition_Value,DEMIL_Code,DEMIL_IC,Ship_Date,total_value
Ship_Date,,,,,,,,,,,
2001-10-02,KS,GOODLAND POLICE DEPT,5855-01-461-8169,"VIEWER KIT,NIGHT VISION",1,Kit,4943.40,D,1,2001-10-02,4943.40
2009-10-13,WA,KENT POLICE DEPT,1005-00-017-9546,"HANDLE ASSEMBLY,CHARGING",2,Assembly,16.45,D,1,2009-10-13,32.90
2009-10-20,WA,ISSAQUAH POLICE DEPT,8465-01-416-4626,"SPECTACLES KIT,BALLISTIC AND LASER PROTECTIVE",1,Kit,15.16,D,1,2009-10-20,15.16
2009-10-24,GA,MORVEN POLICE DEPT,2320-01-411-4393,"KIT,COMBAT IDENTIFI",8,Kit,1231.00,D,1,2009-10-24,9848.00
2009-10-28,KY,LOUISVILLE METRO POLICE DEPT,8465-01-416-4626,"SPECTACLES KIT,BALLISTIC AND LASER PROTECTIVE",45,Kit,15.16,D,1,2009-10-28,682.20
...,...,...,...,...,...,...,...,...,...,...,...
2021-09-29,AZ,ARIZONA DEPT OF PUBLIC SAFETY,8430-01-516-1682,"BOOTS,COMBAT",3,Pair,101.78,A,1,2021-09-29,305.34
2021-09-29,AZ,ARIZONA DEPT OF PUBLIC SAFETY,8430-01-516-1608,"BOOTS,COMBAT",1,Pair,101.78,A,1,2021-09-29,101.78
2021-09-29,CA,LOS ANGELES CSD,7520-01-383-7950,"MARKER ASSORTMENT,TUBE TYPE",23,Set,5.63,A,1,2021-09-29,129.49


In [28]:
# Ensuring the integrity of total_value does not need changed due to UI status (does not appear so)

In [29]:
def wrangle_1033():
    '''
    This function takes our 1033 data and creates a dataframe to then prepare.
    It fills the Nulls, corrects datatypes, sets index to the Ship_Date, and creates
    a better representation of total assets given away with the 'total_value' column.
    '''
    # Read the local csv to a dataframe
    df = pd.read_csv('dod_all_states.csv')
    # Change spacing in columns for ease of use
    df.columns = df.columns.str.replace(' ','_')
    # Filling NaN for DEMIL_IC columns
    df.DEMIL_IC = df.DEMIL_IC.fillna(1)
    # Changing Ship_Date to datetime format and setting as index
    df.Ship_Date = pd.to_datetime(df.Ship_Date)
    df = df.set_index(df.Ship_Date)
    df = df.sort_index()
    # changing DEMIL_IC to proper datetype
    df.DEMIL_IC = df.DEMIL_IC.astype(int)
    df.DEMIL_IC = df.DEMIL_IC.astype(object)
    # Feature engineering 'total_value' column to better represent acquisition values
    df['total_value'] = df.Quantity * df.Acquisition_Value
    # Dropping 'Station_Type' column, deemed unnecessary
    df = df.drop(columns='Station_Type')
    # Return the df
    return df